<a href="https://colab.research.google.com/github/nasilemakdirajo/working_hours/blob/main/Working%20Hours.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===== Upload your file in Colab (or replace with a file path if running locally) =====
try:
    from google.colab import files
    up = files.upload()
    FNAME = next(iter(up.keys()))
    RAW = up[FNAME]
except Exception:
    # If not in Colab, read from disk:
    FNAME = "Attendance Summary_001_08.XLS"
    with open(FNAME, "rb") as f:
        RAW = f.read()

import re, csv, io
import xml.etree.ElementTree as ET
from datetime import date

# ---------- Config ----------
MORNING_CUTOFF_MIN = 6 * 60   # OUT 00:01..05:59 counts as next day
LEFT_WIDTH = 8                # Date, Week, then 6 time cells (IN/OUT x3)
RIGHT_OFFSET = 8              # Right block starts 8 columns after left
DEFAULT_YEAR = 2025           # Used if year cannot be inferred from header
OUT_CSV = "attendance_totals_all_with_times.csv"
# ----------------------------

# ---------- Encoding + XML helpers ----------
def sniff_xml_encoding(b: bytes) -> str:
    head = b[:200].decode("ascii", errors="ignore")
    m = re.search(r'encoding\s*=\s*["\']([^"\']+)["\']', head, re.I)
    enc = (m.group(1).lower() if m else "utf-8")
    # Normalize a few common names
    if enc in ("utf-16", "utf16"): return "utf-16"
    if enc in ("utf-16le", "utf16le"): return "utf-16le"
    if enc in ("utf-16be", "utf16be"): return "utf-16be"
    if enc in ("utf-8", "utf8"): return "utf-8-sig"
    if enc in ("gb2312", "gbk", "gb18030"): return "gb18030"
    return enc

def read_spreadsheetml_grid(raw: bytes):
    enc = sniff_xml_encoding(raw)
    text = raw.decode(enc, errors="replace")
    ns = {"ss": "urn:schemas-microsoft-com:office:spreadsheet"}
    root = ET.fromstring(text)
    ws = root.find(".//ss:Worksheet", ns)
    if ws is None:
        raise ValueError("No <Worksheet> found.")
    table = ws.find(".//ss:Table", ns)
    if table is None:
        raise ValueError("No <Table> found.")
    grid = []
    for r in table.findall("ss:Row", ns):
        row, cidx = [], 1
        for c in r.findall("ss:Cell", ns):
            idx_attr = c.get("{urn:schemas-microsoft-com:office:spreadsheet}Index")
            if idx_attr:
                idx = int(idx_attr)
                while cidx < idx:
                    row.append("")
                    cidx += 1
            d = c.find("ss:Data", ns)
            txt = "" if d is None or d.text is None else str(d.text)
            row.append(txt)
            cidx += 1
        grid.append(row)
    return grid, text

# ---------- Parsing helpers ----------
PLUS_RE = re.compile(r"^\s*(\d{1,2}):(\d{2})\s*\+\s*(\d{1,2}):(\d{2})\s*$")

def clean_time(s):
    if s is None: return ""
    s = str(s).strip().replace("\u3000", " ").replace("\u00A0", " ")
    for ch in ("：", "，", "．", ",", "."):
        s = s.replace(ch, ":")
    return s

def parse_time_to_min(s):
    """Return minutes since 00:00 (0..1439) or None. Supports '00:00+1:48'."""
    s = clean_time(s)
    if not s: return None
    m = PLUS_RE.match(s)
    if m:
        h1, m1, h2, m2 = map(int, m.groups())
        return (h1 % 24) * 60 + (m1 % 60) + (h2 * 60 + m2)
    if ":" not in s: return None
    try:
        h, m = s.split(":", 1)
        return (int(h) % 24) * 60 + (int(m) % 60)
    except:
        return None

def total_minutes_from_times(times6):
    """
    Pair IN -> next OUT across the 6 cells in order (IN/OUT x3).
    OUT rules:
      - 00:00 => 24:00 (same day)
      - 00:01..05:59 => next day (add 24h)
    """
    total = 0
    stack = []
    for i, val in enumerate(times6):
        t = parse_time_to_min(val)
        if t is None:
            continue
        is_in = (i % 2 == 0)
        if is_in:
            stack.append(t)
        else:
            if not stack:
                continue
            inm = stack.pop()
            if t == 0:
                outm = 1440
            elif 0 < t < MORNING_CUTOFF_MIN:
                outm = t + 1440
            else:
                outm = t
                if outm < inm:
                    outm += 1440
            dur = outm - inm
            if dur < 0:
                dur += 1440
            total += dur
    return total

def find_dual_header_row(grid, start_idx=0):
    for i in range(start_idx, len(grid)):
        r = [str(x).strip() for x in grid[i]]
        if len(r) >= LEFT_WIDTH + RIGHT_OFFSET and r[0] == "Date" and r[1] == "Week" \
           and r[RIGHT_OFFSET] == "Date" and r[RIGHT_OFFSET + 1] == "Week":
            return i
    return None

def take_block_row(row, base_idx):
    vals = row + [""] * (base_idx + LEFT_WIDTH - len(row))
    return {
        "Date": vals[base_idx + 0].strip(),
        "Week": vals[base_idx + 1].strip(),
        "Times": [vals[base_idx + j].strip() for j in range(2, LEFT_WIDTH)],
    }

def year_hint_from_header_text(txt: str) -> int:
    m = re.search(r"Date:(\d{2})\.\d{2}\.\d{2}", txt)
    return 2000 + int(m.group(1)) if m else DEFAULT_YEAR

def mmdd_to_date(mmdd: str, year_hint: int):
    s = (mmdd or "").strip()
    if not s: return None
    if "." in s:
        mm, dd = s.split(".", 1)
    elif "/" in s:
        mm, dd = s.split("/", 1)
    else:
        return None
    try:
        return date(year_hint, int(mm), int(dd))
    except Exception:
        return None

def date_to_ddmmyyyy(dt: date) -> str:
    return f"{dt.day:02d}/{dt.month:02d}/{dt.year:04d}"

# ---------- Read file ----------
is_xml = RAW.lstrip().startswith(b"<?xml") and b"urn:schemas-microsoft-com:office:spreadsheet" in RAW
if not is_xml:
    raise RuntimeError("This script expects the SpreadsheetML (.XLS XML) export. Please upload that file.")

grid, raw_text = read_spreadsheetml_grid(RAW)

# ---------- Process all employees ----------
rows_out = []
i = 0
while i < len(grid):
    # Find the next dual header
    header_row = find_dual_header_row(grid, i)
    if header_row is None:
        break

    # Try to find Name:xxxx above the header (within a few rows)
    name = "UNKNOWN"
    year_hint = year_hint_from_header_text(raw_text)
    for seek in range(max(0, header_row - 8), header_row + 1):
        for cell in [str(x) for x in grid[seek]]:
            if cell.startswith("Name:"):
                name = cell.split("Name:", 1)[1].strip()
            if cell.startswith("Date:"):
                m = re.search(r"Date:(\d{2})\.\d{2}\.\d{2}", cell)
                if m:
                    year_hint = 2000 + int(m.group(1))

    # Collect rows for this employee (until the first blank-date row)
    r = header_row + 1
    person_rows = []
    while r < len(grid):
        row = [str(x) if x is not None else "" for x in grid[r]]
        left = take_block_row(row, 0)
        right = take_block_row(row, RIGHT_OFFSET)
        if not left["Date"] and not right["Date"]:
            r += 1
            break  # end of this employee section

        if left["Date"]:
            dt = mmdd_to_date(left["Date"], year_hint)
            person_rows.append(("L", dt, left["Week"], left["Times"]))
        if right["Date"]:
            dt = mmdd_to_date(right["Date"], year_hint)
            person_rows.append(("R", dt, right["Week"], right["Times"]))
        r += 1

    # Sort by calendar date so B2=01/08 -> B3=02/08, etc.
    person_rows = [x for x in person_rows if x[1] is not None]
    person_rows.sort(key=lambda t: t[1])

    # Emit combined rows including raw times & total
    for _, dt, wk, times6 in person_rows:
        total_min = total_minutes_from_times(times6)
        hh, mm = divmod(total_min, 60)
        out = {
            "Name": name,
            "Date": date_to_ddmmyyyy(dt),
            "Week": wk,
            "Morning (IN)": times6[0],
            "Morning (OUT)": times6[1],
            "Afternoon (IN)": times6[2],
            "Afternoon (OUT)": times6[3],
            "Overtime (IN)": times6[4],
            "Overtime (OUT)": times6[5],
            "Total_hh:mm": f"{hh}:{mm:02d}",
        }
        rows_out.append(out)

    # Continue searching after this employee's block
    i = r + 1

# ---------- Write output ----------
fieldnames = [
    "Name","Date","Week",
    "Morning (IN)","Morning (OUT)",
    "Afternoon (IN)","Afternoon (OUT)",
    "Overtime (IN)","Overtime (OUT)",
    "Total_hh:mm"
]
with open(OUT_CSV, "w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=fieldnames)
    w.writeheader()
    w.writerows(rows_out)

print(f"Done. Wrote {OUT_CSV} with {len(rows_out)} rows.")
try:
    files.download(OUT_CSV)  # works in Colab
except Exception:
    pass

Saving Attendance Summary_001_08.XLS to Attendance Summary_001_08 (2).XLS
Done. Wrote attendance_totals_all_with_times.csv with 279 rows.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>